# 使用朴素贝叶斯过滤垃圾邮件

1. 收集数据：提供的文本文件。
2. 准备数据：将文本文件解析成词条向量。
3. 分析数据：检查词条确保解析的正确性。
4. 训练算法：使用trainNB0()函数。
5. 测试算法：使用classifyNB()，并且构建一个新的测试函数来计算文档集的错误率。
6. 使用算法：构建一个完整的程序对一组文档进行分类，将错分的文档输出到屏幕上。

## 准备数据：切分文本

In [1]:
mySent = 'This book is the best book on Python or M.L. I hava ever laid eyes upon.'

In [2]:
mySent.split()

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'hava',
 'ever',
 'laid',
 'eyes',
 'upon.']

### 修正标点符号的切分

In [3]:
# 引入正则表达式模块
import re

In [4]:
# 匹配字符串
regEx = re.compile('\W+')
listOfTokens = regEx.split(mySent)

In [5]:
listOfTokens

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M',
 'L',
 'I',
 'hava',
 'ever',
 'laid',
 'eyes',
 'upon',
 '']

### 将单词转换为小写，以便统一词袋

In [6]:
[tok.lower() for tok in listOfTokens if len(tok) > 0]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'hava',
 'ever',
 'laid',
 'eyes',
 'upon']

### 对数据集中一封完整电子邮件的实际处理结果

In [7]:
emailText = open('email/ham/6.txt').read()

In [8]:
listOfTokens = regEx.split(emailText)

## 测试算法：使用朴素贝叶斯进行交叉验证

### 文件解析及完整的垃圾邮件测试函数

In [9]:
def textParse(bigString):
    import re
    listOfTokens = re.split('\W+', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [10]:
# 创建一个包含在所有文档中出现的不重复词的列表
def createVocabList(dataSet):
    # 创建一个空集合，为了去重
    vocabSet = set([])
    for document in dataSet:
        # 创建两个集合的并集
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

In [11]:
def setOfWords2Vec(vocabList, inputSet):
    # 创建一个其中所含元素都是0的向量，进行该向量数组的初始化
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            # 1或0表示词汇表中的单词在输入文档中是否出现
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word )
    return returnVec

In [12]:
import numpy as np
from math import log
# trainMatrix-文档矩阵，trainCategory-由每篇文档类别标签所构成的向量
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    # 求出有侮辱性词语的文档占比
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    # 初始化概率，num为分子，denom为分母
    # 如果其中一个概率值为0， 那么最后的乘积也为0，为了降低这种影响，
    # 可以将所有词的出现数初始化为1， 并将分母初始化为2
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        # p1表示侮辱性词语，p0表示正常词语
        if trainCategory[i] == 1:
            # 向量相加
            # 侮辱性词语的词频数向量
            p1Num += trainMatrix[i]
            # 侮辱性词语的总词数
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # 对每个元素做除法，求出该词的概率即p(w|c)
    # 解决下溢出问题：利用ln(a*b)=ln(a)+ln(b)
    p1Vect = np.array([log(x) for x in p1Num/p1Denom])
    p0Vect = np.array([log(x) for x in p0Num/p0Denom])
    return p0Vect, p1Vect, pAbusive

In [13]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # 元素相乘，相当于p(w0|c1)p(w1|c1)...
    # 由于p(w)是一样的，所以在求p1的时候就不需要做除法运算了，计算p(c|w)
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [14]:
def spamTest():
    import random
    docList = []; classList = []; fullText = []
    for i in range(1, 26):
        # 采用ISO-8859-1的编码读取文件
        wordList = textParse(open('email/spam/%d.txt' % i, encoding='ISO-8859-1').read())
        docList.append(wordList)
        fullText.append(wordList)
        # 标记为1
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i, encoding='ISO-8859-1').read())
        docList.append(wordList)
        fullText.extend(wordList)
        # 标记为0
        classList.append(0)
    # 创建一个包含在所有文档中出现的不重复词的列表
    vocabList = createVocabList(docList)
    # 由于每一类有25个文本，故一共有50个文本
    trainingSet = range(50); testSet = []
    # 随机取10个作为测试集，剩下的40个文本作为训练集
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(list(trainingSet)[randIndex])
    trainMat = []; trainClasses = []
    # 构建初始化训练集的数据矩阵和特征值
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    # 训练算法
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    # 测试算法
    for docIndex in testSet:
        # 将测试集数据进行向量化
        wordVecotr = setOfWords2Vec(vocabList, docList[docIndex])
        # 进行测试，如果发现错误，则错误数加1
        if classifyNB(np.array(wordVecotr), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    # 打印出总的错误百分比
    print('the error rate is: ', float(errorCount)/len(testSet))

In [15]:
spamTest()

the error rate is:  0.0
